# Vision Tranformer 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import ViTForImageClassification, ViTFeatureExtractor
import matplotlib.pyplot as plt

# Set paths
data_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'
base_image_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/'

# Load the dataset
df = pd.read_csv(data_path)
df = df.dropna(subset=['Surgery diagnosis in number'])  # Drop rows with NaN labels

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Load the ViT feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 as required by ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

# Custom Dataset class for loading images and labels
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx]['Surgery diagnosis in number'])  # Ensure label is integer

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)  # Label as long for CrossEntropyLoss

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize ViT model for binary classification
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=2)
model = model.to(device)

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Tracking metrics
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []
val_aucs = []

# Training loop with tqdm
num_epochs = 100
best_val_auc = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_predictions = 0

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images).logits
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            correct_predictions += (predictions == labels).sum().item()

            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    # Calculate training metrics
    train_loss /= len(train_loader)
    train_acc = correct_predictions / len(train_dataset)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_labels = []
    val_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images).logits
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            predictions = torch.softmax(outputs, dim=1)[:, 1]
            val_correct_predictions += (torch.argmax(outputs, dim=1) == labels).sum().item()

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predictions.cpu().numpy())

    # Calculate validation metrics
    val_loss /= len(test_loader)
    val_acc = val_correct_predictions / len(test_dataset)
    val_auc = roc_auc_score(val_labels, val_preds)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    val_aucs.append(val_auc)

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}')

    # Save the model with the best AUC score
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), 'best_ViT_Test.pth')
        print(f"Model saved with best validation AUC: {best_val_auc:.4f}")

# Plot Loss and Accuracy over Epochs
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()


testing


In [ ]:
model.load_state_dict(torch.load('/home/iambrink/NOH_Thyroid_Cancer_Data/Notebooks/best_ViT_Test.pth'))
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        predictions = torch.argmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Calculate final metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auc_score = roc_auc_score(y_true, y_pred)

# Print final evaluation metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

# Plotting metrics
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

# ResNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Set paths
data_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'
base_image_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/'

# Load the dataset
df = pd.read_csv(data_path)
df = df.dropna(subset=['Surgery diagnosis in number'])  # Drop rows with NaN labels

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Define image transformations for ResNet preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
])

# Custom Dataset class for loading images and labels with ResNet preprocessing
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx]['Surgery diagnosis in number'])  # Ensure label is integer

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)  # Label as long for CrossEntropyLoss

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize ResNet model for binary classification with two output labels
device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.resnet50(pretrained=True)  # Use ResNet-50 pretrained model
model.fc = nn.Linear(model.fc.in_features, 2)  # Modify final layer for binary classification
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for binary classification

# Tracking lists for loss, accuracy, and AUC
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
val_aucs = []

# Training loop with tqdm
num_epochs = 100
best_val_auc = 0.0  # Track the best validation AUC

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_predictions = 0

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)  # Get outputs for ResNet
            loss = criterion(outputs, labels)  # Use logits directly

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update training metrics
            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)  # Get class predictions
            correct_predictions += (predictions == labels).sum().item()

            # Update progress bar
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    # Calculate average training loss and accuracy
    train_loss /= len(train_loader)
    train_losses.append(train_loss)  # Append training loss
    train_acc = correct_predictions / len(train_dataset)
    train_accuracies.append(train_acc)  # Append training accuracy

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_labels = []
    val_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            predictions = torch.softmax(outputs, dim=1)[:, 1]  # Probability for class 1
            val_correct_predictions += (torch.argmax(outputs, dim=1) == labels).sum().item()

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predictions.cpu().numpy())

    # Calculate validation loss, accuracy, and AUC score
    val_loss /= len(test_loader)
    val_losses.append(val_loss)  # Append validation loss
    val_acc = val_correct_predictions / len(test_dataset)
    val_accuracies.append(val_acc)  # Append validation accuracy
    val_auc = roc_auc_score(val_labels, val_preds)
    val_aucs.append(val_auc)  # Append AUC

    # Print metrics
    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}')

    # Save model if AUC improves
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), 'best_ResNet_AUC_Model.pth')
        print(f"Model saved with best validation AUC: {best_val_auc:.4f}")

# Final evaluation
model.load_state_dict(torch.load('best_ResNet_AUC_Model.pth'))
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)  # Get the predicted class label

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Calculate final metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auc_score = roc_auc_score(y_true, y_pred)

# Print final evaluation metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

# Plotting metrics
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()


# Basic CNN model with no specified archetecture

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from tqdm import tqdm
from sklearn.model_selection import train_test_split

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  # Assuming RGB images
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.fc1 = nn.Linear(32* 64 *64,128) # 256px
        self.fc1 = nn.Linear(32 * 61 * 61, 128) # for 244 px
        # self.fc1 = nn.Linear(32 * 80 * 80, 128)  #ffor 320 px Adjust this based on the output size
        self.fc2 = nn.Linear(128, 1)  # Binary classification (output one value)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms import v2
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from torchvision.transforms import v2
from tqdm import tqdm
# Set paths
data_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'
base_image_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/'

# Load the dataset using ImageFolder
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(244, antialias=True),
    v2.RandomCrop(244),
    v2.RandomVerticalFlip(p=0.5),
    v2.RandomHorizontalFlip(p=0.5),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create your own Dataset class to load images and labels
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['Surgery diagnosis in number']

        if self.transform:
            image = self.transform(image)

        return image, label

# Load data and prepare for training
df = pd.read_csv(data_path)
df = df.dropna(subset=['Surgery diagnosis in number'])  # Drop rows with NaN labels

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()  # For binary classification


num_epochs = 100
# Initialize best AUC variable
best_val_auc = 0.0  # To keep track of the best validation AUC
# Initialize lists to store loss and accuracy values
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
val_aucs = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_predictions = 0

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.float().to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update training metrics
            train_loss += loss.item()
            predictions = torch.round(torch.sigmoid(outputs.squeeze()))
            correct_predictions += (predictions == labels).sum().item()

            # Update progress bar
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    # Calculate average training loss and accuracy for the epoch
    train_loss /= len(train_loader)
    train_acc = correct_predictions / len(train_dataset)

    # Append training loss and accuracy
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_labels = []
    val_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels.float().to(device))

            val_loss += loss.item()
            predictions = torch.sigmoid(outputs.squeeze())  # Get probabilities
            val_correct_predictions += (torch.round(predictions) == labels.float().to(device)).sum().item()

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predictions.cpu().numpy())

    # Calculate average validation loss, accuracy, and AUC for the epoch
    val_loss /= len(test_loader)
    val_acc = val_correct_predictions / len(test_dataset)
    val_auc = roc_auc_score(val_labels, val_preds)

    # Append validation metrics
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    val_aucs.append(val_auc)

    # Print metrics for this epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}')

    # Save the model if validation AUC improves
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), 'best_CNN_AUC_Model.pth')
        print(f"Model saved with best validation AUC: {best_val_auc:.4f}")

# Plot Loss and Accuracy over Epochs
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

print("Training and Validation plots generated.")
# Final evaluation
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd

# Ensure the best trained model is loaded
model.load_state_dict(torch.load('/home/iambrink/NOH_Thyroid_Cancer_Data/best_CNN_AUC_Model.pth'))  # Load the best model saved during training
model.eval()

# Initialize lists for true labels and predictions
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        predictions = torch.round(torch.sigmoid(outputs.squeeze()))  # Convert to binary predictions
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Calculate additional metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

# Print additional metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {roc_auc_score(y_true, y_pred):.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot Confusion Matrix and Classification Report
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

# Confusion Matrix Heatmap
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax[0])
ax[0].set_title("Confusion Matrix")
ax[0].set_xlabel("Predicted Labels")
ax[0].set_ylabel("True Labels")
ax[0].set_xticklabels(['Non-Cancer', 'Cancer'])
ax[0].set_yticklabels(['Non-Cancer', 'Cancer'])

# Generate and plot Classification Report as a DataFrame
report_df = pd.DataFrame(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer'], output_dict=True)).transpose()

sns.heatmap(report_df.iloc[:-1, :].T, annot=True, cmap='YlGnBu', cbar=False, ax=ax[1], fmt='.2f')
ax[1].set_title("Classification Report")
ax[1].set_xlabel("Metrics")
ax[1].set_ylabel("Classes")

plt.tight_layout()
plt.show()

print("Testing completed!")



# Efficinet-B0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Set paths
data_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'
base_image_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/'

# Load the dataset
df = pd.read_csv(data_path)
df = df.dropna(subset=['Surgery diagnosis in number'])  # Drop rows with NaN labels

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Define image transformations for ResNet preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
])

# Custom Dataset class for loading images and labels with ResNet preprocessing
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx]['Surgery diagnosis in number'])  # Ensure label is integer

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)  # Label as long for CrossEntropyLoss

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize ResNet model for binary classification with two output labels
device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.efficientnet_b0(pretrained=True)  # Use EfficientNet-B0 pretrained model
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)  # Modify final layer for binary classification
model = model.to(device)

# Optimizer and criterion remain the same
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for binary classification

# Tracking lists for loss, accuracy, and AUC
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
val_aucs = []

# Training loop with tqdm
num_epochs = 100
best_val_auc = 0.0  # Track the best validation AUC

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_predictions = 0

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)  # Get outputs for ResNet
            loss = criterion(outputs, labels)  # Use logits directly

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update training metrics
            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)  # Get class predictions
            correct_predictions += (predictions == labels).sum().item()

            # Update progress bar
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    # Calculate average training loss and accuracy
    train_loss /= len(train_loader)
    train_losses.append(train_loss)  # Append training loss
    train_acc = correct_predictions / len(train_dataset)
    train_accuracies.append(train_acc)  # Append training accuracy

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_labels = []
    val_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            predictions = torch.softmax(outputs, dim=1)[:, 1]  # Probability for class 1
            val_correct_predictions += (torch.argmax(outputs, dim=1) == labels).sum().item()

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predictions.cpu().numpy())

    # Calculate validation loss, accuracy, and AUC score
    val_loss /= len(test_loader)
    val_losses.append(val_loss)  # Append validation loss
    val_acc = val_correct_predictions / len(test_dataset)
    val_accuracies.append(val_acc)  # Append validation accuracy
    val_auc = roc_auc_score(val_labels, val_preds)
    val_aucs.append(val_auc)  # Append AUC

    # Print metrics
    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}')

    # Save model if AUC improves
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), 'EB0_Test_64_BS.pth')
        print(f"Model saved with best validation AUC: {best_val_auc:.4f}")

# Final evaluation
model.load_state_dict(torch.load('EB0_Test_64_BS.pth'))
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)  # Get the predicted class label

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Calculate final metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auc_score = roc_auc_score(y_true, y_pred)

# Print final evaluation metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

# Plotting metrics
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()


# DeiT (Data efficient image Transformer)

In [ ]:
import torch
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader
from transformers import DeiTForImageClassification, DeiTFeatureExtractor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Set paths
data_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'
base_image_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/'

# Load the dataset
df = pd.read_csv(data_path)
df = df.dropna(subset=['Surgery diagnosis in number'])  # Drop rows with NaN labels

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Initialize the DeiT feature extractor
feature_extractor = DeiTFeatureExtractor.from_pretrained("facebook/deit-base-distilled-patch16-224")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

# Custom Dataset class for DeiT
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx]['Surgery diagnosis in number'])

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize DeiT model for binary classification
device = "cuda" if torch.cuda.is_available() else "cpu"
model = DeiTForImageClassification.from_pretrained(
    "facebook/deit-base-distilled-patch16-224",
    num_labels=2  # Set for binary classification
)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Training and validation metrics
train_losses, val_losses, train_accuracies, val_accuracies, val_aucs = [], [], [], [], []
num_epochs = 100
best_val_auc = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_predictions = 0

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images).logits  # Access logits for DeiT
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update training metrics
            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            correct_predictions += (predictions == labels).sum().item()

            # Update progress bar
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    # Calculate training metrics
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_acc = correct_predictions / len(train_dataset)
    train_accuracies.append(train_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_labels, val_preds = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images).logits
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            predictions = torch.softmax(outputs, dim=1)[:, 1]
            val_correct_predictions += (torch.argmax(outputs, dim=1) == labels).sum().item()

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predictions.cpu().numpy())

    # Calculate validation metrics
    val_loss /= len(test_loader)
    val_losses.append(val_loss)
    val_acc = val_correct_predictions / len(test_dataset)
    val_accuracies.append(val_acc)
    val_auc = roc_auc_score(val_labels, val_preds)
    val_aucs.append(val_auc)

    # Print metrics
    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}')

    # Save model if AUC improves
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), 'best_DeiT_AUC_Model.pth')
        print(f"Model saved with best validation AUC: {best_val_auc:.4f}")

# Final evaluation
model.load_state_dict(torch.load('best_DeiT_AUC_Model.pth'))
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        predictions = torch.argmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Calculate final metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auc_score = roc_auc_score(y_true, y_pred)

# Print final evaluation metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

# Plotting metrics
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()


# SWIN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import timm  # For using Swin Transformer models

# Set paths
data_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'
base_image_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/'

# Load the dataset
df = pd.read_csv(data_path)
df = df.dropna(subset=['Surgery diagnosis in number'])  # Drop rows with NaN labels

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Define image transformations for Swin Transformer preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for Swin Transformer
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization
])

# Custom Dataset class for loading images and labels
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx]['Surgery diagnosis in number'])  # Ensure label is integer

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize Swin Tiny Transformer for binary classification
device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=2)
model = model.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss for binary classification

# Tracking lists for loss, accuracy, and AUC
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
val_aucs = []

# Training loop with tqdm
num_epochs = 100
best_val_auc = 0.0  # Track the best validation AUC

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_predictions = 0

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            correct_predictions += (predictions == labels).sum().item()

            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_acc = correct_predictions / len(train_dataset)
    train_accuracies.append(train_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_labels = []
    val_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            predictions = torch.softmax(outputs, dim=1)[:, 1]
            val_correct_predictions += (torch.argmax(outputs, dim=1) == labels).sum().item()

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predictions.cpu().numpy())

    val_loss /= len(test_loader)
    val_losses.append(val_loss)
    val_acc = val_correct_predictions / len(test_dataset)
    val_accuracies.append(val_acc)
    val_auc = roc_auc_score(val_labels, val_preds)
    val_aucs.append(val_auc)

    # Print metrics
    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}')

    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), 'best_SwinTiny_AUC_Model.pth')
        print(f"Model saved with best validation AUC: {best_val_auc:.4f}")

# Final evaluation
model.load_state_dict(torch.load('best_SwinTiny_AUC_Model.pth'))
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Calculate and print final evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auc_score = roc_auc_score(y_true, y_pred)

print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

# Plotting metrics
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()


# Evaluate Test_loader from Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Set paths
data_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'
base_image_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/'

# Load the dataset
df = pd.read_csv(data_path)
df = df.dropna(subset=['Surgery diagnosis in number'])  # Drop rows with NaN labels

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Define image transformations for ResNet preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
])

# Custom Dataset class for loading images and labels with ResNet preprocessing
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx]['Surgery diagnosis in number'])  # Ensure label is integer

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)  # Label as long for CrossEntropyLoss

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



test transormer models

In [ ]:
from transformers import DeiTForImageClassification
import torch
import torch
import torch.nn.functional as F
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Initialize DeiT model for binary classification
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=2)
model.load_state_dict(torch.load('/home/iambrink/NOH_Thyroid_Cancer_Data/best_ViT_AUC_Model.pth'))  # Load the trained weights for ViT
model = model.to(device)
model.eval()
model.eval()

# Evaluation
y_true = []
y_pred = []
y_prob = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get model predictions
        outputs = model(images).logits
        predictions = torch.argmax(outputs, dim=1)  # Predicted class labels
        probabilities = F.softmax(outputs, dim=1)[:, 1]  # Probability of class 1 (Cancer)

        # Store labels, predictions, and probabilities
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        y_prob.extend(probabilities.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=1)
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
auc_score = roc_auc_score(y_true, y_prob)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Print additional metrics 
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

# Calculate and plot confusion matrix and classification report as image
conf_matrix = confusion_matrix(y_true, y_pred)
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

# Confusion Matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax[0])
ax[0].set_title("Confusion Matrix")
ax[0].set_xlabel("Predicted Labels")
ax[0].set_ylabel("True Labels")
ax[0].set_xticklabels(['Non-Cancer', 'Cancer'])
ax[0].set_yticklabels(['Non-Cancer', 'Cancer'])

# Classification Report as Image
report_df = pd.DataFrame(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer'], output_dict=True)).transpose()
sns.heatmap(report_df.iloc[:-1, :].T, annot=True, cmap='YlGnBu', cbar=False, ax=ax[1], fmt='.2f')
ax[1].set_title("Classification Report")
ax[1].set_xlabel("Metrics")
ax[1].set_ylabel("Classes")

plt.tight_layout()
plt.show()


In [ ]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd

# Ensure the best trained model is loaded
model.load_state_dict(torch.load('/home/iambrink/NOH_Thyroid_Cancer_Data/best_cnn_model.pth'))  # Load the best model saved during training
model.eval()

# Initialize lists for true labels and predictions
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        predictions = torch.round(torch.sigmoid(outputs.squeeze()))  # Convert to binary predictions
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Calculate additional metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

# Print additional metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {roc_auc_score(y_true, y_pred):.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot Confusion Matrix and Classification Report
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

# Confusion Matrix Heatmap
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax[0])
ax[0].set_title("Confusion Matrix")
ax[0].set_xlabel("Predicted Labels")
ax[0].set_ylabel("True Labels")
ax[0].set_xticklabels(['Non-Cancer', 'Cancer'])
ax[0].set_yticklabels(['Non-Cancer', 'Cancer'])

# Generate and plot Classification Report as a DataFrame
report_df = pd.DataFrame(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer'], output_dict=True)).transpose()

sns.heatmap(report_df.iloc[:-1, :].T, annot=True, cmap='YlGnBu', cbar=False, ax=ax[1], fmt='.2f')
ax[1].set_title("Classification Report")
ax[1].set_xlabel("Metrics")
ax[1].set_ylabel("Classes")

plt.tight_layout()
plt.show()

print("Testing completed!")


# Testing Data

Preload

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import ViTForImageClassification, ViTFeatureExtractor
import matplotlib.pyplot as plt

# Set paths
data_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'
base_image_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/'

# Load the dataset
df = pd.read_csv(data_path)
df = df.dropna(subset=['Surgery diagnosis in number'])  # Drop rows with NaN labels

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Define image transformations for EfficientNet-B0 preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for EfficientNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for EfficientNet
])

# Custom Dataset class for loading images and labels with EfficientNet preprocessing
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx]['Surgery diagnosis in number'])  # Ensure label is integer

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)  # Label as long for CrossEntropyLoss

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Efficinet

In [ ]:
import torch
import random
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# Set the seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for deterministic results

# Load the saved model checkpoint
model = models.efficientnet_b0(pretrained=False)  # Initialize EfficientNet-B0 model
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)  # Modify the final layer
model.load_state_dict(torch.load('/home/iambrink/NOH_Thyroid_Cancer_Data/best_EfficientNet_AUC_Model_NOH_ONLY.pth'))  # Load the trained weights
model = model.to(device)  # Move model to GPU if available
model.eval()  # Set the model to evaluation mode

# Lists to store true labels and predictions
y_true = []
y_pred = []
y_prob = []

# Disable gradient calculation for faster inference
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get model predictions
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)  # Predicted class labels
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1 (cancer)

        # Store labels and predictions
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        y_prob.extend(probabilities.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=1)  # For binary classification, use `pos_label=1` for Cancer
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
auc_score = roc_auc_score(y_true, y_prob)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Print additional metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")


DeiT

In [ ]:
import torch
import random
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
from transformers import DeiTForImageClassification  # Import DeiT model

# Set the seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for deterministic results

# Load the saved DeiT model checkpoint
model = DeiTForImageClassification.from_pretrained('facebook/deit-base-distilled-patch16-224', num_labels=2)
model.load_state_dict(torch.load('/home/iambrink/NOH_Thyroid_Cancer_Data/best_DeiT_AUC_Model_TAN_ONLY.pth'))  # Load the trained weights
model = model.to(device)  # Move model to GPU if available
model.eval()  # Set the model to evaluation mode

# Lists to store true labels and predictions
y_true = []
y_pred = []
y_prob = []

# Disable gradient calculation for faster inference
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get model predictions
        outputs = model(images).logits  # Use `.logits` to get the output logits from DeiT
        predictions = torch.argmax(outputs, dim=1)  # Predicted class labels
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1 (Cancer)

        # Store labels and predictions
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        y_prob.extend(probabilities.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=1)  # For binary classification, use `pos_label=1` for Cancer
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
auc_score = roc_auc_score(y_true, y_prob)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Print additional metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")


SWIN

In [ ]:
import torch
import random
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import timm

# Set the seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for deterministic results

# Initialize device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the Swin Tiny model
model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=2)  # Initialize Swin Tiny with binary classification
model.load_state_dict(torch.load('/home/iambrink/NOH_Thyroid_Cancer_Data/best_Swin_AUC_Model_NOH_ONLY.pth'))  # Load the trained weights
model = model.to(device)
model.eval()  # Set model to evaluation mode

# Lists to store true labels and predictions
y_true = []
y_pred = []
y_prob = []

# Disable gradient calculation for faster inference
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get model predictions
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)  # Predicted class labels
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1 (Cancer)

        # Store labels and predictions
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        y_prob.extend(probabilities.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=1)  # For binary classification, use `pos_label=1` for Cancer
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
auc_score = roc_auc_score(y_true, y_prob)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Print additional metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")


resnet

In [ ]:
import torch
import random
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import timm

# Set the seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for deterministic results

# Initialize device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the ResNet-50 model
model = timm.create_model('resnet50', pretrained=True, num_classes=2)  # Initialize ResNet-50 with binary classification
model.load_state_dict(torch.load('best_ResNet_AUC_Model_TAN_ONLY.pth'))  # Load the trained weights for ResNet-50
model = model.to(device)
model.eval()  # Set model to evaluation mode

# Lists to store true labels and predictions
y_true = []
y_pred = []
y_prob = []

# Disable gradient calculation for faster inference
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get model predictions
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)  # Predicted class labels
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1 (Cancer)

        # Store labels and predictions
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        y_prob.extend(probabilities.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=1)  # For binary classification, use `pos_label=1` for Cancer
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
auc_score = roc_auc_score(y_true, y_prob)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Print additional metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")


ViT

In [ ]:
import torch
import random
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
from transformers import ViTForImageClassification

# Set the seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for deterministic results

# Initialize device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the ViT model
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=2)
model.load_state_dict(torch.load('/home/iambrink/NOH_Thyroid_Cancer_Data/best_ViT_AUC_Model.pth'))  # Load the trained weights for ViT
model = model.to(device)
model.eval()  # Set model to evaluation mode

# Lists to store true labels and predictions
y_true = []
y_pred = []
y_prob = []

# Disable gradient calculation for faster inference
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get model predictions
        outputs = model(images).logits  # .logits to access the raw predictions
        predictions = torch.argmax(outputs, dim=1)  # Predicted class labels
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1 (Cancer)

        # Store labels and predictions
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        y_prob.extend(probabilities.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=1)  # For binary classification, use `pos_label=1` for Cancer
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
auc_score = roc_auc_score(y_true, y_prob)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Print additional metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")


simple CNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchvision.transforms import v2

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  # Assuming RGB images
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.fc1 = nn.Linear(32* 64 *64,128) # 256px
        self.fc1 = nn.Linear(32 * 61 * 61, 128) # for 244 px
        # self.fc1 = nn.Linear(32 * 80 * 80, 128)  #ffor 320 px Adjust this based on the output size
        self.fc2 = nn.Linear(128, 1)  # Binary classification (output one value)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x
# Load the dataset using ImageFolder
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(244, antialias=True),
    v2.RandomCrop(244),
    v2.RandomVerticalFlip(p=0.5),
    v2.RandomHorizontalFlip(p=0.5),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create your own Dataset class to load images and labels
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['Surgery diagnosis in number']

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
import torch
import random
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import torch.nn as nn

# Set the seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for deterministic results

train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Initialize device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the simple CNN model
model = SimpleCNN().to(device)
model.load_state_dict(torch.load('/home/iambrink/NOH_Thyroid_Cancer_Data/best_CNN_AUC_Model_NOH_ONLY.pth'))
model.eval()  # Set model to evaluation mode

# Lists to store true labels and predictions
y_true = []
y_pred = []
y_prob = []

# Disable gradient calculation for faster inference
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get model predictions
        outputs = model(images).squeeze()  # Get outputs and remove extra dimension
        probabilities = torch.sigmoid(outputs)  # Probability for class 1 (Cancer)
        predictions = (probabilities >= 0.5).int()  # Binary predictions based on threshold

        # Store labels and predictions
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        y_prob.extend(probabilities.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=1)  # For binary classification, use `pos_label=1` for Cancer
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
auc_score = roc_auc_score(y_true, y_prob)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Cancer', 'Cancer']))

# Print additional metrics
print("\nFinal Evaluation Metrics:")
print(f"  ROC-AUC Score: {auc_score:.4f}")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")
